# ![](https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png) Webscraping Project 4 Lab

Week 4 | Day 4

In this project, we will practice two major skills: collecting data by scraping a website and then building a binary predictor with Logistic Regression.

We are going to collect salary information on data science jobs in a variety of markets. Then using the location, title and summary of the job we will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information (as you will see in this exercise), being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

Normally, we could use regression for this task; however, we will convert this problem into classification and use Logistic Regression.

- Question: Why would we want this to be a classification problem?
- Answer: While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range be may be useful.

Therefore, the first part of the assignment will be focused on scraping Indeed.com (or other sites at your team's discretion). In the second part, the focus is on using listings with salary information to build a model and predict high or low salaries and what features are predictive of that result.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Setup a request (using `requests`) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)

In [14]:
import requests
import bs4
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium import webdriver
import datetime
import urllib

In [ ]:
## YOUR CODE HERE
import requests
import bs4
from bs4 import BeautifulSoup
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium import webdriver

dcap = dict(DesiredCapabilities.PHANTOMJS)
dcap["phantomjs.page.settings.userAgent"] = (
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36"
)

driver = webdriver.PhantomJS(executable_path='/Applications/anaconda/anaconda/bin/phantomJS', desired_capabilities=dcap)
driver.set_window_size(1024, 768) 

URL = "http://www.indeed.com/jobs?q=data+scientist&l=New+York&start=20"

driver.get(URL)
page_html = driver.page_source
soup = BeautifulSoup(page_html, "lxml")
print(soup.prettify())

Let's look at one result more closely. A single `result` looks like

```
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&amp;campaignid=serp-linkcompanyname&amp;fromjk=2480d203f7e97210&amp;jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some more verbose elements removed, we can see that there is some structure to the above:
- The salary is available in a `nobr` element inside of a `td` element with `class='snip`.
- The title of a job is in a link with class set to `jobtitle` and a `data-tn-element="jobTitle`.  
- The location is set in a `span` with `class='location'`. 
- The company is set in a `span` with `class='company'`. 

### Write 4 functions to extract each item: location, company, job, and salary.

example: 
```python
def extract_location_from_result(result):
    return result.find ...
```


- Make sure these functions are robust and can handle cases where the data/field may not be available.
- Test the functions on the results above
- Include any other features you may want to use later (e.g. summary, #of reviews...)

Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.

- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the `l=New+York` and the `start=10`. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).

### Complete the following code to collect results from multiple cities and starting points. 
- Indeed.com only has salary information for an estimated 20% of job postings. You may want to add other cities to the list below to gather more data. 
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different

In [ ]:
################################################################
############### EDIT THESE CONSTANTS
############### 
###############        EDIT THESE CONSTANTS

MAX_RESULTS_PER_CITY = 1000      ### DO NOT SET MORE THAN 1000
URL_SEARCH_TERM = 'Data Scientist' ### DO NOT SET MORE THAN SINGLE SEARCH TERM (TITLE)
CITY_SET = ['New York', 'Chicago', 'San Francisco', 'Austin', 'Atlanta', '', 'Boston', 'Seattle'\
            'Los Angeles','Washington, DC', 'San Jose','Denver', 'Atlanta','Houston',\
            'Dallas','Nashville','San Diego','Cleveland','Minneapolis','Baltimore','Philadelphia','Detroit']
PHANTOM_PATH = '/Applications/anaconda/anaconda/bin/phantomJS'
###############
################################################################


import requests
import bs4
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium import webdriver
import datetime
import urllib

def extract_location_from_resultRow(result):
    try:
        location = (result.find(class_='location').text.strip())
    except:
        location = ''
    return location

def extract_company_from_resultRow(result):
    try:
        company = (result.find(class_='company').text.strip())
    except:
        company = ''
    return company

def extract_jkid_from_resultRow(result):
    try:
        row = (result.find(class_='jobtitle turnstileLink'))
        jkid = result['data-jk']
    except: 
        jkid = ''
    return jkid

def extract_title_from_resultRow(result):
    try:
        title = (result.find(class_='turnstileLink'))
        title_text = title.text
    except: 
        title_text = ''
    return title_text

def extract_salary_from_resultRow(result):
    try:
        salary = (result.find(class_='snip').find('nobr').text)
    except:
        salary = ''
    salary_text = salary
    return salary_text

def extract_reviews_from_resultRow(result):
    try:
        reviews = (result.find(class_='slNoUnderline').text.strip().strip(' reviews').replace(',',''))
    except: 
        reviews = ''
    return reviews

def extract_stars_from_resultRow(result):
    try: 
        stars = (result.find(class_='rating')['style']).split(';background-position:')[1].split(':')[1].split('px')[0].strip()
    except: dro
        stars = ''
    return stars

def extract_date_from_resultRow(result):
    try: 
        date = (result.find(class_='date').text.strip(' ago').strip())
    except: 
        date = ''
    return date

def extract_summary_from_resultRow(result):
    try: 
        summary = (result.find("span", {"itemprop" : "description"}).text.strip())
    except: 
        summary = ''
    return summary

dcap = dict(DesiredCapabilities.PHANTOMJS)
dcap["phantomjs.page.settings.userAgent"] = ("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36")

driver = webdriver.PhantomJS(executable_path=PHANTOM_PATH, desired_capabilities=dcap)
driver.set_window_size(1024, 768) 

for city in CITY_SET:
    job_dict = []
    now = datetime.datetime.now()
    for start in range(0, MAX_RESULTS_PER_CITY, 10):

        URL = "http://www.indeed.com/jobs?q="+urllib.quote(URL_SEARCH_TERM)+"&l="+urllib.quote(city)+"&start="+str(start)
        driver.get(URL)
        soup = BeautifulSoup(driver.page_source, "lxml")

        for i in soup.findAll("div", {"data-tn-component" : "organicJob"}):

            location = extract_location_from_resultRow(i)
            company = extract_company_from_resultRow(i)
            summary = extract_summary_from_resultRow(i)
            jkid = extract_jkid_from_resultRow(i)
            title = extract_title_from_resultRow(i)
            salary = extract_salary_from_resultRow(i)
            reviews = extract_reviews_from_resultRow(i)
            stars = extract_stars_from_resultRow(i)
            post_date = extract_date_from_resultRow(i)

            job_dict.append([location, company, summary, jkid, title, salary, stars, reviews, post_date, now])
            
        job_df = pd.DataFrame(job_dict, columns=['location', 'company', 'summary', 'jkid', 'title', 'salary', 'stars', 'reviews', 'post_date', 'pull_date'])       

    job_df.to_csv('scrape'+city+'_'+str(MAX_RESULTS_PER_CITY)+'.csv', encoding='utf-8')
        

#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

In [33]:
master_df = pd.read_csv('master.csv')

# DELETE ANY HEADER ROWS LEFT OVER FROM CSV MERGE
try: master_df = master_df[master_df['reviews'] != 'reviews'] 
except: pass

In [34]:
###### REVIEWS CLEAN TO FLOAT
######

master_df['reviews'] = master_df['reviews'].fillna(0)

def indeed_review_cleanup(review): 
    try:
        review = review.str.replace(',','')
        review = review.strip(' reviews')
        review = review.strip(' review')
        review = review.strip('reviews')
        review = review.strip()
        review = float(review)
    except:
        #print review
        pass
    return review

master_df['clean_review'] = master_df[['reviews']].applymap(lambda x:indeed_review_cleanup(x))

master_df['clean_review'].sort_values().unique()

master_df['clean_review'] = master_df['clean_review'].astype(float)
master_df['reviews'] = master_df['clean_review']
master_df.drop('clean_review', axis=1, inplace=True)

#########  END CLEAN REVIEWS
###################

In [35]:
###### POST_DATE CLEAN TO FLOAT
######

try:
    master_df['clean_post_date'] = master_df['post_date']
except: pass


def post_date_to_day_float(dateValue):
    try:
        temp = dateValue
        dateValue.replace('s','')
        if 'day' in dateValue:
            temp = dateValue.split()[0]
        elif 'hour' in dateValue:
            temp = dateValue.split()[0]
            temp = float(temp)/24
        elif 'minute' in dateValue:
            temp = dateValue.split()[0]
            temp = float(temp)/24/60
        if '+' in dateValue:
            temp = 45           
    except: 
        pass
    return temp

master_df['clean_post_date'] = master_df[['clean_post_date']].applymap(lambda x: post_date_to_day_float(x))

master_df['clean_post_date'].sort_values().unique()

master_df['clean_post_date'] = master_df['clean_post_date'].astype(float)
master_df['post_date'] = master_df['clean_post_date']
master_df.drop('clean_post_date', axis=1, inplace=True)
master_df.rename(columns = {'post_date':'post_date_daysAgo'}, inplace=True)

#########  END CLEAN POST_DATE
###################

In [36]:
###### STARS CLEAN TO FLOAT
######


master_df['clean_stars'] = master_df['stars'].fillna(0)
master_df['clean_stars'] = master_df[['stars']].astype(float).applymap(lambda x: x//6/2)


master_df['stars'] = master_df['clean_stars']
master_df.drop('clean_stars', axis=1, inplace=True)


#########  END CLEAN STARS
###################

In [37]:
#####Create JOB_LINK column from JKID
#####

master_df['job_link'] = master_df[['jkid']].applymap(lambda x: 'http://www.indeed.com/rc/clk?jk='+x)

#########  END JOB_LINK COLUMN
###################

In [38]:
##### Location Cleanup
#####

def location_cleanup(location):
    temp = location
    temp_city = location.split(',')[0]
    try:
        temp_state = location.split(',')[1].split()[0]
    except: 
        temp_state = ''
    return temp_city+", "+temp_state
    
master_df['location_clean'] = master_df[['location']].applymap(lambda x: location_cleanup(x))
master_df['location_clean'].sort_values().unique()

master_df['location'] = master_df['location_clean']
master_df.drop('location_clean', axis=1, inplace=True)

#########  END LOCATION CLEANUP COLUMN
###################

In [39]:
##### Salary Cleanup
#####

master_df['salary'] = master_df['salary'].fillna(0)

def cleanup_salary(salary):
    if "year" in str(salary):
        temp = salary.strip(" a year")
        temp = temp.split('-')
        low_range = int(temp[0].strip().replace("$","").replace(",",""))
        high_range = int(temp[-1].strip().replace("$","").replace(",",""))
        avg = (low_range + high_range) / 2
        salary_list = [low_range,high_range,avg]
    elif "month" in str(salary):
        temp = salary.replace("a month","")
        temp = temp.split('-')
        low_range = int(temp[0].replace("$","").replace(",",""))*12
        high_range = int(temp[-1].replace("$","").replace(",",""))*12
        avg = (low_range + high_range) / 2
        salary_list = [low_range,high_range,avg]
    elif "hour" in str(salary):
        temp = salary.replace("an hour","")
        temp = temp.split('-')
        low_range = float(temp[0].replace("$","").replace(",",""))*2080
        high_range = float(temp[-1].replace("$","").replace(",",""))*2080
        avg = (low_range + high_range) / 2
        salary_list = [low_range,high_range,avg]
    else:
        salary_list = [0,0,0]
        low_range = 0
        high_range = 0
        avg = 0
        
    return low_range, high_range, avg

master_df['salary_clean'] = master_df[['salary']].applymap(lambda x: cleanup_salary(x))

master_df['salary'] = master_df['salary_clean']

master_df['sal_low'] = master_df['salary'].apply(lambda x: x[0])
master_df['sal_high'] = master_df['salary'].apply(lambda x: x[1])
master_df['sal_avg'] = master_df['salary'].apply(lambda x: x[2])

master_df.drop('salary_clean', axis=1, inplace=True)
master_df.drop('salary', axis=1, inplace=True)



#########  END SALARY CLEANUP COLUMN
###################

In [40]:
master_df = master_df.reset_index(drop=True)
#Cleans title
master_df['title'] = map(lambda x: x.lower(), master_df['title'])
master_df['title'] = (master_df['title'].replace( 'sr.','senior ', regex=True))
master_df['title'] = (master_df['title'].replace( 'senior  ','senior ', regex=True))

In [41]:
#Remove title from database
temp_df = master_df.T
temp_df = temp_df[[ind for ind, x in enumerate(master_df['title']) if "title" not in x]].T
master_df = temp_df.reset_index(drop=True)

In [42]:
#Remove Intern from database
temp_df = master_df.T
temp_df = temp_df[[ind for ind, x in enumerate(master_df['title']) if "intern" not in x]].T
master_df = temp_df.reset_index(drop=True)

In [43]:
#Remove Devop from database
temp_df = master_df.T
temp_df = temp_df[[ind for ind, x in enumerate(master_df['title']) if "devop" not in x]].T
master_df = temp_df.reset_index(drop=True)

In [44]:
df_range = len(master_df)
master_df['level'] = 0

In [28]:
#Entry = 0
#Middle = 1
#Senior = 2
# for x in range(df_range):
#     if "associate" in master_df['title'][x] or "junior" in master_df['title'][x] or "entry" in master_df['title'][x]:
#         master_df['level'][x] = 0
#     elif "senior" in master_df['title'][x] or "principal" in master_df['title'][x] or "lead" in master_df['title'][x]:
#         master_df['level'][x] = 2
#     else:  
#         master_df['level'][x] = 1

def bin_titles(title):
    if any(title 'associate' in title:
        return 0
    elif 'junior' in title: 
        return 0
    elif 'entry' in title: 
        return 0
    elif 'senior' in title: 
        return 2
    elif 'principal' in title: 
        return 2
    elif 'lead' in title: 
        return 2
    else: 
        return 1
    
        
master_df['level'] = master_df[['title']].applymap(lambda x: bin_titles(x))


In [31]:
master_df['level'].value_counts()

1    9009
2    2785
0     975
Name: level, dtype: int64

In [45]:

mean_salary = master_df[master_df['sal_avg'] != 0]['sal_avg'].to_frame().mean()
median_salary = master_df[master_df['sal_avg'] != 0]['sal_avg'].to_frame().median()

print master_df[master_df['sal_avg'] != 0].shape[0]

print 'mean {} median {}'.format(mean_salary[-1], median_salary[-1])



558
mean 104725.185663 median 97500.0


In [ ]:



has_salary = master_df[master_df['salary'] != (0,0,0)].shape[0]
all_records = master_df.shape[0]
print "Job listings with salary info:", has_salary
print "Total job listings: ", all_records
print "Salaried listings / Total listings", round((float(has_salary) / all_records) * 100, 3), '%'
master_df.head(5)
master_df['title'].sort_values().unique()


# data['recipe'] = data['title'].str.lower().str.contains('recipe')
# data['electronic'] = data['title'].str.lower().str.contains('electronic')
# data['tips'] = data['title'].str.lower().str.contains('tips')

# stacked = pd.DataFrame(master_df['summary'].str.split().tolist()).stack()
# final = pd.DataFrame(stacked.value_counts())
# final.reset_index(inplace=True)
# final['unique'] = final['index'].sort_values().unique()
# final['unique']
# import nltk
# final['tagged'] = final[['index']].applymap(lambda x: nltk.pos_tag(x.strip()))
# final.info()
master_df

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer(
    lowercase = True,
    binary=True,  # Create binary features
    stop_words='english', # Ignore common words such as 'the', 'and'
    max_features=50, # Only use the top 50 most common words
)


# This builds a matrix with a row per website (or data point) and column per word (using all words in the dataset)
X = v.fit_transform(master_df.summary).todense()
X = pd.DataFrame(X, columns=v.get_feature_names())
print X.columns.tolist()
X.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer(
    lowercase = True, 
    binary=True,  # Create binary features
    stop_words='english', # Ignore common words such as 'the', 'and'
    max_features=50, # Only use the top 50 most common words
)


# This builds a matrix with a row per website (or data point) and column per word (using all words in the dataset)
X = v.fit_transform(master_df.title).todense()
X = pd.DataFrame(X, columns=v.get_feature_names())
print X.columns.tolist()
X.head()

In [ ]:
stacked = pd.DataFrame(master_df['summary'].str.split().tolist()).stack()
import string
punct = set(string.punctuation)

final = pd.DataFrame(stacked.value_counts())
print final.info()
final.reset_index(inplace=True)
final['unique'] = final['index'].sort_values().unique()
final['unique']


In [ ]:
master_df

In [ ]:
## YOUR CODE HERE

Lastly, we need to clean up salary data. 
1. Some of the salaries are not yearly but hourly, these will be useful to us for now
2. The salaries are given as text and usually with ranges.

#### Filter out the salaries that are not yearly (filter those that refer to hour)

In [ ]:
## YOUR CODE HERE

#### Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary

In [ ]:
## YOUR CODE HERE

### Save your results as a CSV

In [ ]:
## YOUR CODE HERE